In [24]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go 
import matplotlib as plt
from plotly.subplots import make_subplots

In [9]:
data = yf.download(tickers='DELTA.BK', period='max', interval='1d')
data.columns = data.columns.droplevel(1)

[*********************100%***********************]  1 of 1 completed


In [10]:
data = data[data['High'] != data['Low']]
data.reset_index(inplace=True)
data.tail(10)

Price,Date,Close,High,Low,Open,Volume
6091,2025-03-24,68.25,70.00,67.00,70.00,10181500
6092,2025-03-25,70.00,70.50,67.50,69.00,19384600
6093,2025-03-26,73.50,74.75,70.25,70.25,28478600
6094,2025-03-27,72.25,72.75,71.00,72.50,13648300
6095,2025-03-28,69.00,71.50,68.75,71.50,12954900
6096,2025-03-31,65.75,67.25,65.50,65.75,22623900
6097,2025-04-01,66.75,67.00,66.25,67.00,7164700
6098,2025-04-02,67.50,69.00,67.00,67.25,8580900
6099,2025-04-03,60.50,65.50,59.00,65.00,26206000
6100,2025-04-04,56.75,59.50,56.25,59.50,27236900


In [11]:
def rejection_signal(df):
    #Bullish signal
    if (df.Open.iloc[-1] < df.Close.iloc[-1] and
        df.High.iloc[-1] - df.Close.iloc[-1] < abs(df.Open.iloc[-1] - df.Close.iloc[-1])/10 and
        df.Close.iloc[-1] - df.Low.iloc[-1] > abs(df.Open.iloc[-1] - df.Close.iloc[-1])*5):
        return 2
    
    #Bearish signal
    elif (df.Open.iloc[-1] > df.Close.iloc[-1] and
          df.High.iloc[-1] - df.Open.iloc[-1] > abs(df.Open.iloc[-1] - df.Close.iloc[-1])/10 and
          df.Close.iloc[-1] - df.Low.iloc[-1] < abs(df.Open.iloc[-1] - df.Close.iloc[-1])*5):
        return 1
    
    else:
        return 0
    
def engulfing_signal(df):
    previous_candle = df.iloc[-2]
    current_candle = df.iloc[-1]
    
    #Bullish Engulfing
    if ((current_candle['Close'] > previous_candle['Open']) and
        (current_candle['Open'] < previous_candle['Close']) and
        (previous_candle['Open']) > previous_candle['Close']):
        return 2
    
    #Bearish Engulfing
    elif ((current_candle['Open'] > previous_candle['Close']) and
          (current_candle['Close'] < previous_candle['Open']) and
          (previous_candle['Close']) > previous_candle['Open']):
        return 1
    
    else:
        return 0
    
signal = [0]*len(data)
for i in range(3, len(data)):
    df = data[i-3 : i+1]
    signal[i] = rejection_signal(df)
data['Rejection_signal'] = signal

signal = [0]*len(data)
for i in range(1, len(data)):
    df = data[i-1 : i+1]
    signal[i] = engulfing_signal(df)
data['engulfing_signal'] = signal

In [18]:
up_count = 0
down_count = 0
total_count = 0

for i in range(len(data) - 1):
    if data.engulfing_signal.iloc[i] == 1:
        total_count += 1
        if data.Close.iloc[i+1] > data.Open.iloc[i+1]:
            up_count += 1
        elif data.Close.iloc[i+1] < data.Open.iloc[i+1]:
            down_count += 1
            
if total_count > 0:
    up_percentage = (up_count / total_count) * 100
    down_percentage = (down_count / total_count) * 100
else:
    up_percentage = 0
    down_percentage = 0
    print('No bearish engulfing signals found in the dataset.')

In [17]:
print(f'up_percentage\t : {up_percentage}')
print(f'down_percentage\t : {down_percentage}')
print(f'Total count\t : {total_count}')

up_percentage	 : 45.45454545454545
down_percentage	 : 47.4025974025974
Total count	 : 154


In [19]:
data[data['engulfing_signal'] == 1]

Price,Date,Close,High,Low,Open,Volume,Rejection_signal,engulfing_signal
56,2000-03-23,-0.752506,-0.761731,-0.740146,-0.740146,3610045,0,1
130,2000-07-18,-1.111274,-1.125701,-1.068773,-1.068773,4739305,0,1
140,2000-08-01,-1.104256,-1.104256,-1.032900,-1.032900,257215,0,1
190,2000-10-12,-0.651948,-0.651948,-0.616465,-0.623483,12138315,0,1
214,2000-11-16,-0.968953,-0.975972,-0.933470,-0.940489,4750250,0,1
...,...,...,...,...,...,...,...,...
6017,2024-12-03,150.158051,152.146899,148.666414,151.649687,8944600,1,1
6030,2024-12-24,150.158051,153.638535,149.660838,152.146899,16441300,1,1
6065,2025-02-14,112.369934,116.347631,111.375510,115.850419,16660000,1,1
6067,2025-02-18,77.316483,86.514907,77.067876,86.514907,50749700,0,1


In [22]:
st = 0
dfpl = data[st:st + 150].copy()
fig = go.Figure(data = go.Candlestick(x=df['Date'],
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name='OHLC'))

fig.show()